<a href="https://colab.research.google.com/github/JossueE/Piper-Octybot-Voice/blob/main/Piper/Piper_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0) Colab runtime + GPU check

In [1]:
# Check GPU
import torch, platform, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

Python: 3.12.12
PyTorch: 2.8.0+cu126
CUDA available: True
Tue Oct 28 23:10:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |              

1) System packages (incl. eSpeak dev)

In [2]:
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake ninja-build espeak-ng espeak-ng-data libespeak-ng-dev pkg-config ffmpeg
!pkg-config --modversion espeak-ng

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,086 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,827 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,070 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/u

2) Clone repo fresh

In [3]:
%cd /content
!rm -rf piper1-gpl
!git clone https://github.com/OHF-voice/piper1-gpl.git
%cd piper1-gpl
!pwd

/content
Cloning into 'piper1-gpl'...
remote: Enumerating objects: 785, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 785 (delta 160), reused 148 (delta 148), pack-reused 590 (from 2)
Receiving objects: 100% (785/785), 4.69 MiB | 11.43 MiB/s, done.
Resolving deltas: 100% (438/438), done.
/content/piper1-gpl
/content/piper1-gpl


3) Python deps (editable install, no venv in Colab)

In [7]:
!python3 -m pip uninstall -y ipython
!python3 -m pip install "ipython==7.34.0" "jedi>=0.16"
!python3 -m pip check


Found existing installation: ipython 9.6.0
Uninstalling ipython-9.6.0:
  Successfully uninstalled ipython-9.6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 12.7 MB/s  0:00:00
No broken requirements found.


In [8]:
!python3 -m pip install --upgrade pip setuptools wheel
!python3 -m pip install -e ".[train]"

Obtaining file:///content/piper1-gpl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for piper-tts (pyproject.toml) ... done
  Created wheel for piper-tts: filename=piper_tts-1.3.1-0.editable-py3-none-any.whl size=14401 sha256=076cf3beef7a492336e9d9ce7b7a673e640d114cb54d205fba27b3dddffa14b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-i9c1ogo4/wheels/22/38/4e/d9fd0ab8ba964eb8332fc89984f961285397f819ab17291146
Successfully built piper-tts
  Attempting uninstall: piper-tts
    Found existing installation: piper-tts 1.3.1
    Uninstalling piper-tts-1.3.1:
      Successfully uninstalled piper-tts-1.3.1


4) Build the Cython extension used for alignment

In [9]:
%cd /content/piper1-gpl
!chmod +x ./build_monotonic_align.sh
!./build_monotonic_align.sh

/content/piper1-gpl
Compiling /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
/usr/local/lib/python3.12/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
performance hint: core.pyx:7:5: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:38:6: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
P

5) Dev build (repo mode)

In [10]:
!python3 -m pip install --upgrade pip setuptools wheel scikit-build cmake ninja

  Using cached scikit_build-0.18.1-py3-none-any.whl.metadata (18 kB)
  Using cached cmake-4.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (6.5 kB)
  Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (5.1 kB)
Using cached scikit_build-0.18.1-py3-none-any.whl (85 kB)
Using cached cmake-4.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (29.7 MB)
Using cached ninja-1.13.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (180 kB)
  Attempting uninstall: cmake
    Found existing installation: cmake 3.31.6
    Uninstalling cmake-3.31.6:
      Successfully uninstalled cmake-3.31.6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [cmake]


In [11]:
%cd /content/piper1-gpl
!python3 setup.py build_ext --inplace -v

/content/piper1-gpl


--------------------------------------------------------------------------------
-- Trying 'Ninja' generator
--------------------------------
---------------------------
----------------------
-----------------
------------
-------
--
Not searching for unused variables given on the command line.
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX

6) (Optional) Mount Google Drive for datasets and outputs

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


7) Set paths and training hyperparams

In [17]:
from pathlib import Path

# ==== CHANGE THESE ====
VOICE_NAME      = "Octybot"
ESPEAK_VOICE    = "es-419"  # run `!espeak-ng --voices` to see options
SAMPLE_RATE_HZ  = 22050
BATCH_SIZE      = 32       # drop to 8 or 4 if you OOM

DATA_ROOT       = Path("/content/drive/MyDrive/")
AUDIO_DIR       = DATA_ROOT / "data"
CSV_PATH        = DATA_ROOT / "metadata.csv"

CACHE_DIR       = Path("/content/piper_cache")
CONFIG_PATH     = DATA_ROOT / f"{VOICE_NAME}.json"

# Optional: start from an existing checkpoint to speed up & stabilize training
# Get a .ckpt from https://huggingface.co/datasets/rhasspy/piper-checkpoints (medium quality recommended)
CKPT_PATH       = ""  # e.g., "/content/drive/MyDrive/piper_ckpts/en_US-lessac-medium.ckpt"

# Make sure dirs exist
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print("CSV exists:", CSV_PATH.exists())
print("Audio dir exists:", AUDIO_DIR.exists())
print("Cache dir:", CACHE_DIR)
print("Config will be written to:", CONFIG_PATH)

CSV exists: True
Audio dir exists: True
Cache dir: /content/piper_cache
Config will be written to: /content/drive/MyDrive/Octybot.json


8) Quick sanity checks

In [16]:
!espeak-ng --voices | head -n 40

Pty Language       Age/Gender VoiceName          File                 Other Languages
 5  af              --/M      Afrikaans          gmw/af               
 5  am              --/M      Amharic            sem/am               
 5  an              --/M      Aragonese          roa/an               
 5  ar              --/M      Arabic             sem/ar               
 5  as              --/M      Assamese           inc/as               
 5  az              --/M      Azerbaijani        trk/az               
 5  ba              --/M      Bashkir            trk/ba               
 5  bg              --/M      Bulgarian          zls/bg               
 5  bn              --/M      Bengali            inc/bn               
 5  bpy             --/M      Bishnupriya_Manipuri inc/bpy              
 5  bs              --/M      Bosnian            zls/bs               
 5  ca              --/M      Catalan            roa/ca               
 5  cmn             --/M      Chinese_(Mandarin) sit/cmn    

In [24]:
import pandas as pd, io, os, textwrap

csv_path = str(CSV_PATH)
if os.path.exists(csv_path):
    # Read as pipe-delimited, two columns
    try:
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])
        print(df.head())
        # Check a few audio files exist
        missing = [a for a in df["audio"].head(5) if not (AUDIO_DIR/str(a)).exists()]
        print("Missing among first 5:", missing)
    except Exception as e:
        print("CSV read error:", e)
else:
    print("CSV not found at:", csv_path)

  audio                                               text
0  utt0   la Dalila  continuaba anclada bajo el Castill...
1  utt1          la Dalila  quizá permanezca en el puerto:
2  utt2   Algo es algo dijo un calvo al encontrar un peine
3  utt3                              ?Habrá visitado ella?
4  utt4                           A Dios lo que es de Dios
Missing among first 5: ['utt0', 'utt1', 'utt2', 'utt3', 'utt4']


In [23]:
import pandas as pd
import os

csv_path = str(CSV_PATH)

if os.path.exists(csv_path):
    try:
        # Read as pipe-delimited, two columns
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])

        # Remove .wav extension from the 'audio' column
        df['audio'] = df['audio'].str.replace('.wav', '', regex=False)

        # Save the modified DataFrame back to the same CSV file
        df.to_csv(csv_path, sep="|", header=False, index=False)

        print("Successfully removed '.wav' from audio filenames and saved the updated metadata.csv")
        print(df.head())

    except Exception as e:
        print("Error processing or saving CSV:", e)
else:
    print("CSV not found at:", csv_path)

Successfully removed '.wav' from audio filenames and saved the updated metadata.csv
  audio                                               text
0  utt0   la Dalila  continuaba anclada bajo el Castill...
1  utt1          la Dalila  quizá permanezca en el puerto:
2  utt2   Algo es algo dijo un calvo al encontrar un peine
3  utt3                              ?Habrá visitado ella?
4  utt4                           A Dios lo que es de Dios


9) Kick off training

In [25]:
!python3 -m piper.train fit \
  --data.voice_name "Octybot" \
  --data.csv_path "/content/drive/MyDrive/metadata.csv" \
  --data.audio_dir "/content/drive/MyDrive/data" \
  --model.sample_rate 22050 \
  --data.espeak_voice "es-419" \
  --data.cache_dir "/content/piper_cache" \
  --data.config_path "/content/drive/MyDrive/Octybot.json" \
  --data.batch_size 8 \
  --ckpt_path "https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt"

Streaming output truncated to the last 5000 lines.
Restoring states from the checkpoint path at https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt
Lightning automatically upgraded your loaded checkpoint from v1.9.0 to v2.5.5. To apply the upgrade to your files permanently, run `python -m lightning.pytorch.utilities.upgrade_checkpoint https:/huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/en/en_US/lessac/medium/epoch%3D2164-step%3D1355540.ckpt`
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:445: The dirpath has changed from '/home/hansenm/larynx2/local/en-us/lessac/medium/lightning_logs/version_2/checkpoints' to '/content/piper1-gpl/lightning_logs/version_1/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISI

10. Export to onnx

In [ ]:
!python3 -m piper.train.export_onnx \
  --checkpoint "/content/piper1-gpl/lightning_logs/version_2/checkpoints/epoch=2174-step=2680.ckpt" \
  --output-file "/content/model.onnx"

In [ ]:
!cp /content/drive/MyDrive/tts_data/myvoice/my_colab_voice.json /content/model.onnx.json